In [1]:
import pandas as pd
from itertools import combinations
from collections import defaultdict
import csv


In [1]:
import mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd
from tqdm import tqdm
import efficient_apriori
from efficient_apriori import apriori

In [2]:
itemset_item_trn = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\itemset_item_training.csv', names = ['itemset', 'item'])
itemset_item_trn

,itemset,item
0,8571,18506
1,8571,33644
2,8571,32627
3,5725,14855
4,5725,28037
...,...,...
85817,21913,21835
85818,21913,37137
85819,21913,41653
85820,21913,1682


In [3]:
rating = pd.DataFrame({'rating': [1 for i in range(len(itemset_item_trn))]})
itemsetitem_rating = pd.concat([itemset_item_trn, rating], axis=1)
itemsetitem_rating

,itemset,item,rating
0,8571,18506,1
1,8571,33644,1
2,8571,32627,1
3,5725,14855,1
4,5725,28037,1
...,...,...,...
85817,21913,21835,1
85818,21913,37137,1
85819,21913,41653,1
85820,21913,1682,1


In [4]:
itemset_item_rating = itemsetitem_rating.pivot_table('rating', index='itemset', columns='item')
itemset_item_rating

item,0,1,2,3,4,5,6,7,8,9,...,42553,42554,42555,42556,42557,42558,42559,42560,42561,42562
itemset,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
itemset_item_rating.fillna(0, inplace = True)
itemset_item_rating.head()

item,0,1,2,3,4,5,6,7,8,9,...,42553,42554,42555,42556,42557,42558,42559,42560,42561,42562
itemset,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# 결과창의 itemset은 사실 item임. 이 패키지 자체가 이름이 itemset으로 설정되어 있음.
frequent_items = apriori(itemset_item_rating,
                            min_support=0.001,
                            use_colnames=True)
frequent_items.sort_values("support", ascending=False)
frequent_items

c:\Users\a3179\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.001264,(0)
1,0.002934,(231)
2,0.002663,(291)
3,0.001399,(494)
4,0.003340,(706)
...,...,...
256,0.001805,(41739)
257,0.001399,(41844)
258,0.003972,(42035)
259,0.001760,(42089)


In [11]:
rules = association_rules(frequent_items,
                          metric="support",
                          min_threshold=0.001)

In [29]:
rules[(rules["support"] > 0.0000000005)
      & (rules["confidence"] > 0.000000001)
      & (rules["lift"] > 1)].sort_values("confidence", ascending=False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(14985),(24115),0.001896,0.001309,0.001038,0.547619,418.380952,0.001036,2.207633,0.999505
1,(24115),(14985),0.001309,0.001896,0.001038,0.793103,418.380952,0.001036,4.824171,0.998917


In [6]:
# 연관 규칙 분석
frequent_itemsets = apriori(itemset_item_rating.drop('itemset', axis=1), min_support=0.2, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

KeyError: "['itemset'] not found in axis"

In [3]:
# 연관 규칙 분석을 위한 one-hot 인코딩
one_hot = pd.get_dummies(itemsetitem_trn['item'])
itemsetitem_trn.drop('item', axis=1, inplace=True)
itemsetitem_trn = itemsetitem_trn.join(one_hot)

itemsetitem_trn

,itemset,0,1,2,3,4,5,6,7,8,...,42553,42554,42555,42556,42557,42558,42559,42560,42561,42562
0,8571,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8571,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8571,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5725,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5725,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85817,21913,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85818,21913,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85819,21913,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85820,21913,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# 연관 규칙 분석
frequent_items = apriori(itemsetitem_trn.drop('itemset', axis=1), min_support=0.1, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=0.5)
rules

c:\Users\a3179\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


MemoryError: Unable to allocate 3.40 GiB for an array with shape (85822, 42563) and data type bool

In [15]:
# 사용자의 행동 데이터가 포함된 리스트
user_actions = [
    ['사과', '우유'],
    ['우유', '콩나물'],
    ['사과', '우유', '콩나물'],
    ['사과', '콩나물'],
    ['사과', '우유']
]

# 항목들의 이진 표현을 생성하기 위한 전처리
te = TransactionEncoder()
te_ary = te.fit_transform(user_actions)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,사과,우유,콩나물
0,True,True,False
1,False,True,True
2,True,True,True
3,True,False,True
4,True,True,False


In [16]:
itemsetitem_trn

,itemset,0,1,2,3,4,5,6,7,8,...,42553,42554,42555,42556,42557,42558,42559,42560,42561,42562
0,8571,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8571,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8571,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5725,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5725,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85817,21913,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85818,21913,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85819,21913,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85820,21913,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def support(itemset, transactions):
    count = 0
    for transaction in transactions:
        if set(itemset).issubset(transaction):
            count += 1
    return count / len(transactions)

In [28]:
def generate_itemsets(transactions, support_threshold):
    item_counts = defaultdict(int)
    
    # 첫 번째 단계: 아이템의 빈도수 계산
    for transaction in tqdm(transactions):
        for item in transaction:
            item_counts[item] += 1
    
    # 빈발한 아이템 후보 생성
    frequent_items = set(item for item, count in item_counts.items() if count >= support_threshold)
    
    # 두 번째 단계: 빈발한 아이템 집합 생성
    frequent_itemsets = []
    for k in tqdm(range(2, len(frequent_items) + 1)):
        itemset_candidates = combinations(frequent_items, k)
        frequent_itemset = []
        for candidate in itemset_candidates:
            candidate_count = 0
            for transaction in transactions:
                if set(candidate).issubset(transaction):
                    candidate_count += 1
            if candidate_count >= support_threshold:
                frequent_itemset.append(candidate)
        frequent_itemsets.extend(frequent_itemset)
    
    return frequent_itemsets

In [8]:
def generate_association_rules(frequent_itemsets, confidence_threshold, transactions):
    association_rules = []
    for itemset in tqdm(frequent_itemsets):
        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                antecedent = set(antecedent)
                consequent = itemset - antecedent
                confidence = support(itemset) / support(antecedent)
                if confidence >= confidence_threshold:
                    association_rules.append((antecedent, consequent, confidence))
    return association_rules

In [9]:
def recommend_items(user_history, frequent_itemsets, association_rules, N):
    user_items = set(user_history)
    recommended_items = []
    for rule in association_rules:
        antecedent, consequent, confidence = rule
        if antecedent.issubset(user_items) and not consequent.issubset(user_items):
            for item in consequent:
                if item not in user_items:
                    recommended_items.append((item, confidence))
    recommended_items = sorted(recommended_items, key=lambda x: x[1], reverse=True)
    return [item for item, confidence in recommended_items[:N]]

In [14]:
def load_transactions_from_csv(file_path):
    transactions = []
    items = []
    itemsets = []
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            itemset_id, item_id = int(row[0]), int(row[1])  # 사용자 ID와 아이템 ID 추출
            transactions.append((itemset_id, item_id))  # 튜플로 저장
    return transactions

In [15]:
csv_file_path = r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\itemset_item_training.csv'

itemsetitem_trn = load_transactions_from_csv(csv_file_path)
itemsetitem_trn

[(8571, 18506),
 (8571, 33644),
 (8571, 32627),
 (5725, 14855),
 (5725, 28037),
 (5725, 27903),
 (16895, 29962),
 (16895, 32334),
 (16895, 29247),
 (16895, 28189),
 (16895, 32035),
 (9401, 28348),
 (9401, 23532),
 (9401, 4412),
 (26795, 17208),
 (26795, 33199),
 (26795, 30085),
 (26795, 3230),
 (18720, 6651),
 (18720, 16472),
 (18720, 4805),
 (18720, 9701),
 (16851, 31482),
 (16851, 41434),
 (16851, 32888),
 (16851, 24930),
 (352, 37744),
 (352, 33234),
 (352, 27552),
 (352, 41295),
 (1963, 18082),
 (1963, 32135),
 (1963, 5164),
 (1963, 26858),
 (1963, 6651),
 (1726, 10590),
 (1726, 1217),
 (1726, 13610),
 (1726, 5982),
 (2961, 13722),
 (2961, 2472),
 (2961, 31013),
 (2961, 40663),
 (431, 8611),
 (431, 8562),
 (431, 1363),
 (431, 10637),
 (431, 32180),
 (8788, 2122),
 (8788, 24941),
 (8788, 35558),
 (8788, 15718),
 (10881, 35372),
 (10881, 38644),
 (10881, 5765),
 (10881, 16322),
 (12434, 39756),
 (12434, 41061),
 (12434, 28037),
 (15473, 27999),
 (15473, 10686),
 (15473, 19204),
 (154

In [19]:
itemsetitem_trn[0][1]

18506

In [20]:
transactions = []
items = [itemsetitem_trn[0][1]]

for i in range(len(itemsetitem_trn)-1):
    if itemsetitem_trn[i][0] == itemsetitem_trn[i+1][0]:
        items.append(itemsetitem_trn[i+1][1])
    else:
        transactions.append(items)
        items = [itemsetitem_trn[i+1][1]]

transactions

[[18506, 33644, 32627],
 [14855, 28037, 27903],
 [29962, 32334, 29247, 28189, 32035],
 [28348, 23532, 4412],
 [17208, 33199, 30085, 3230],
 [6651, 16472, 4805, 9701],
 [31482, 41434, 32888, 24930],
 [37744, 33234, 27552, 41295],
 [18082, 32135, 5164, 26858, 6651],
 [10590, 1217, 13610, 5982],
 [13722, 2472, 31013, 40663],
 [8611, 8562, 1363, 10637, 32180],
 [2122, 24941, 35558, 15718],
 [35372, 38644, 5765, 16322],
 [39756, 41061, 28037],
 [27999, 10686, 19204, 5184, 35358],
 [24753, 37218, 29989, 573],
 [35634, 34716, 17382],
 [25574, 34505, 39860, 41543],
 [14985, 40541, 29798, 25402, 39787],
 [3219, 37339, 28955, 37770],
 [5195, 16349, 20895, 28189, 21651],
 [7840, 6968, 39768, 14926],
 [33646, 39360, 29508, 40796],
 [33815, 36553, 2926],
 [22811, 41992, 32979, 22624, 22118],
 [22775, 36646, 26324, 1795],
 [8968, 27439, 565],
 [12277, 27891, 39871, 24295, 4986],
 [27567, 25076, 1535, 2335, 21404],
 [2338, 23998, 35320, 25464],
 [42316, 15726, 4978],
 [31220, 34338, 14190, 34909],
 [

In [22]:
csv_file_path = r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\itemset_item_valid_query.csv'

itemsetitem_val_q = load_transactions_from_csv(csv_file_path)
itemsetitem_val_q

[(6998, 28322),
 (6998, 34217),
 (6998, 12058),
 (8941, 10176),
 (8941, 22364),
 (8941, 5654),
 (11717, 35384),
 (11717, 27447),
 (11717, 23115),
 (18595, 10434),
 (18595, 31432),
 (18595, 494),
 (18595, 24743),
 (23064, 12939),
 (23064, 19379),
 (23064, 23604),
 (23064, 6349),
 (12477, 41176),
 (12477, 18616),
 (12477, 9739),
 (12477, 31128),
 (5972, 17599),
 (5972, 11280),
 (5972, 19518),
 (23363, 29213),
 (23363, 23345),
 (14918, 18766),
 (14918, 5567),
 (19018, 40090),
 (19018, 39831),
 (19018, 29057),
 (25730, 19896),
 (25730, 26337),
 (26182, 38939),
 (26182, 35285),
 (13467, 17126),
 (13467, 13144),
 (17056, 26359),
 (17056, 29995),
 (23672, 19547),
 (23672, 28741),
 (23672, 8197),
 (4448, 15682),
 (4448, 40676),
 (16200, 16134),
 (16200, 35774),
 (16200, 7204),
 (16200, 5588),
 (20471, 36773),
 (20471, 36584),
 (20471, 37061),
 (12215, 10434),
 (12215, 31432),
 (12215, 5151),
 (12215, 17262),
 (23893, 36911),
 (23893, 41490),
 (3532, 7636),
 (3532, 1190),
 (17597, 37790),
 (175

In [24]:
val_transactions = []
items = [itemsetitem_val_q[0][1]]

for i in range(len(itemsetitem_val_q)-1):
    if itemsetitem_val_q[i][0] == itemsetitem_val_q[i+1][0]:
        items.append(itemsetitem_val_q[i+1][1])
    else:
        val_transactions.append(items)
        items = [itemsetitem_val_q[i+1][1]]

val_transactions

[[28322, 34217, 12058],
 [10176, 22364, 5654],
 [35384, 27447, 23115],
 [10434, 31432, 494, 24743],
 [12939, 19379, 23604, 6349],
 [41176, 18616, 9739, 31128],
 [17599, 11280, 19518],
 [29213, 23345],
 [18766, 5567],
 [40090, 39831, 29057],
 [19896, 26337],
 [38939, 35285],
 [17126, 13144],
 [26359, 29995],
 [19547, 28741, 8197],
 [15682, 40676],
 [16134, 35774, 7204, 5588],
 [36773, 36584, 37061],
 [10434, 31432, 5151, 17262],
 [36911, 41490],
 [7636, 1190],
 [37790, 10910],
 [23858, 16926],
 [35777, 26961, 20261, 15255],
 [19844, 26597, 35360],
 [41206, 11447],
 [12177, 2908],
 [26625, 6968, 712],
 [41585, 30580, 3306, 19809],
 [19896, 38090, 15886],
 [40309, 7922, 10873, 39519],
 [20569, 20031],
 [30660, 24951, 24263, 24687],
 [17589, 5982, 15393, 32026],
 [10047, 16082, 8110],
 [22266, 4311, 6966],
 [16598, 33752, 35166, 15971],
 [39681, 13593, 35743],
 [11646, 18767, 41420],
 [5122, 39880, 25123, 3843],
 [37284, 24529, 31815],
 [4234, 6391],
 [22163, 13687, 34338, 16669],
 [14985,

In [29]:
# 빈발한 아이템 집합 생성
support_threshold = 2
frequent_items = generate_itemsets(transactions, support_threshold)
frequent_items

  0%|          | 0/13889 [2:25:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# 연관 규칙 생성
confidence_threshold = 0.5
association_rules = generate_association_rules(frequent_items, confidence_threshold, transactions)

In [ ]:
# 추천 아이템 생성
N = 100  # 상위 N개 아이템 추천

val_recommendation = []
for i in range(len(val_transactions)):
    recommended_items = recommend_items(val_transactions[i], frequent_items, association_rules, N)
    val_recommendation.append(recommend_items)

val_recommendation

In [ ]:
val_acc = 0
for i in range(itemsetitem_val_a):
    if itemsetitem_val_a[i] in val_recommendation[i]:
        val_acc += 1

val_acc = val_acc/len(itemsetitem_val_a)
print(f'validation accuracy: {val_acc}')

In [ ]:
val_avg_rank = 0
for i in range(itemsetitem_val_a):
    if itensetitem_val_a[i] in val_recommendation[i]:
        val_avg_rank += val_recommendation[i].index(itemsetitem_val_a[i])
    else:
        val_avg_rank += 101

val_avg_rank = val_avg_rank/len(itemsetitem_val_a)
print(f'average validation rank: {val_avg_rank}')